<a href="https://colab.research.google.com/github/AlexKressner/WS24_Supply_Chain_Optimierung/blob/main/Produktionsplanung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip als Paketmanager
! pip install -q pyscipopt

In [ ]:
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Produktionsplanung

In [ ]:
# Erstellen einer Modellinstanz
scip = Model()

## Indexmengen

In [ ]:
J = ['P1','P2'] # Menge der Produkte
I = ['Maschinen','Rohstoffe','Montage'] # Menge der Produktionsressourcen

## Parameter (Daten)

In [ ]:
c={} # Deckungsbeiträge
c['P1']=15
c['P2']=20

In [ ]:
a={} # Produktionskoeffizienten, a[i,j]
a['Maschinen']={'P1':1,'P2':1}
a['Rohstoffe']={'P1':6,'P2':9}
a['Montage']={'P1':0,'P2':1}

In [ ]:
b={} # Kapazitäten Produktionsfaktoren
b['Maschinen']=100
b['Rohstoffe']=720
b['Montage']=60

## Entscheidungsvariablen

In [ ]:
# Definition der Entscheidungsvariablen
# j=1 -> P1 -> x[P1]
# j=2 -> P2 -> x[P2]
x={}
for j in J:
    x[j] = scip.addVar(vtype='C', lb=0, ub=None, name=j)
print('Entscheidungsvariablen =', scip.getVars())

## Zielfunktion

In [ ]:
# Maximierung der gesamten Deckungsbeiträge
scip.setObjective(quicksum(c[j]*x[j] for j in J), sense="maximize")

## Nebenbedingungen/ Restriktionen

In [ ]:
# i=1=Maschine -> x[P1] + x[P2] <= 100
# i=2=Rohstoffe -> 6x[P1] + 9x[P2] <= 720
# i=3=Endmontage -> x[P2] <= 60
for i in I:
    scip.addCons(quicksum(a[i][j]*x[j] for j in J)<=b[i], name=f"Kapazität_{i}")
print('Nebenbedingungen =', scip.getConss())

## Berechnung der Lösung

In [ ]:
scip.optimize()
# Status des Solvers
status = scip.getStatus()
print(f"Status des Solvers: {status} \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', scip.getObjVal())
    for j in J:
        print(f'{j} =', round(scip.getVal(x[j])))
else:
    print('Problem hat keine Lösung')